# SELENIUM SCRAPPING TWITTER

In [1]:
# Import libraries 
import pandas as pd
import csv
import time
import os
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

### INITIALIZE WEBDRIVER

In [2]:
# Initialize the webdriver for Firefox
driver = webdriver.Firefox()

### LOGIN

#### Username

In [3]:
# Go to the page 
driver.get('https://twitter.com/login')

# Introduce the username
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[name="text"]'))).send_keys('lasminis1234@gmail.com')

# Click accept
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]'))).click()

#### Verification

In [4]:
# check if element visible
element = driver.find_elements(By.CSS_SELECTOR, 'input[name="text"]')
if len(element):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[name="text"]'))).send_keys('user1234122334')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div'))).click()

#### Password

In [64]:
# Introduce the password
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[name="password"]'))).send_keys('User1234.')

# Enter account
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div'))).click()

#### Cookies

In [65]:
# Click decline coockies
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div/div/div/div[2]/div[2]'))).click()

#### Notifications

In [66]:
# check if element visible
element = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div/div/div[2]/div[2]/div/span/span')
if len(element):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div/div/div[2]/div[2]/div/span/span'))).click()

### OFFICIAL ACCOUNTS - MOSSOS

#### Searching

In [18]:
# Search "mossos" in searching bar
mossos = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[data-testid = "SearchBox_Search_Input"]')))
mossos.send_keys('mossos')
mossos.send_keys(Keys.ENTER)

In [19]:
# Click into "Personas" tab
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a'))).click()

In [20]:
# Click into "Mossos" account (first one)
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]'))).click()

#### Getting the info

In [21]:
def get_tweet_data(driver):
    tag = driver.find_element(By.CSS_SELECTOR,'div[data-testid = "User-Name"]').text
    # Manage publi
    if len(tag.split('\n')) == 4:
        UserName, handle, _, Date = tag.split('\n')
        timestamp = driver.find_element(By.CSS_SELECTOR, 'time[datetime]').get_attribute('datetime')
    else:
        UserName = " "
        timestamp = "0001-01-01T00:00:00.000Z"
    # To avoid erros for tweets that do not have text
    element = driver.find_elements(By.CSS_SELECTOR,'div[data-testid = "tweetText"]')
    if len(element):
        Text = element[0].text
    else:
        Text = " "
    RetweetCount = driver.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
    tweet = (UserName, Text, RetweetCount, timestamp)
    return tweet

#### Scroll

In [23]:
# Define los tiempos de pausa entre scrolls
scroll_pause_time_1 = 1
scroll_pause_time_2 = 2

# Define variables útiles
df = []
tweet_ids = set()

# Verifica si hemos llegado al final de la región de scroll rastreando la posición de scroll
last_height = driver.execute_script("return document.body.scrollHeight")

# Define una variable para contar los intentos de desplazamiento sin nuevos tweets
scroll_attempt = 0

while True:
    tweets = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')

    if len(tweets) == 0:
        scroll_attempt += 1
    else:
        scroll_attempt = 0

    # Verifica si se han extraído nuevos tweets
    if scroll_attempt >= 3:
        # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
        break

    for tweet in tweets:
        info = get_tweet_data(tweet)
        print(info[1])
        if info:
            # Obtiene la fecha del tweet
            tweet_date = datetime.strptime(info[3], '%Y-%m-%dT%H:%M:%S.%fZ').date()
            print(tweet_date)
            # Crea un identificador de tweet
            tweet_id = ''.join(info)
            # Para evitar listar los mismos tweets
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                df.append(info)

    # Scrolls helps the page to arrive to the following point where we are interested in extracting tweets again
    for _ in range(3):
            print("SCROLLING")
            driver.execute_script("window.scrollBy(0, 700);")
            time.sleep(scroll_pause_time_1)

    # Wait to load page
    time.sleep(scroll_pause_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    print("new_height", new_height)
    print("last_height", last_height)

    # Verifica si se ha llegado al final de la página
    if new_height == last_height:
        scroll_attempt += 1
        if scroll_attempt >= 3:
            # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
            print("ATTEMPTS FINISHED")
            break
        else:
            # Wait before scrolling again
            time.sleep(scroll_pause_time_2)
    else:
        last_height = new_height
        print("SAME HEIGHTS")
        scroll_attempt = 0

Si t'han insultat, agredit, discriminat o maltractat per la teva identitat o orientació sexual, denuncia-ho. T'escoltem. També pots escriure'ns a   mossos.delictes.odi@gencat.cat https://gen.cat/3rCR8mQ #StopLGTBIfòbia
2023-06-12
Trobem un xai, que feia pocs dies que havia nascut, atrapat en uns matolls al riu Congost a Granollers. El recuperem i el lliurem al seu pastor
2023-06-11
Avui, últim dia de 
@lapatum
, un dels dies forts, i nosaltres seguim amb el dispositiu de seguretat. Gaudeix de la festa amb civisme i, si condueixes, recorda que l'única taxa segura és 0,0 #Patum
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 12881
last_height 12215
SAME HEIGHTS
Si t'han insultat, agredit, discriminat o maltractat per la teva identitat o orientació sexual, denuncia-ho. T'escoltem. També pots escriure'ns a   mossos.delictes.odi@gencat.cat https://gen.cat/3rCR8mQ #StopLGTBIfòbia
2023-06-12
Trobem un xai, que feia pocs dies que havia nascut, atrapat en uns matolls al riu Congost a Granol

Iniciem els primers canvis organitzatius de la nova estructura  que permetran desplegar el decret d’estructura d'una policia més moderna i tecnològica. Nous lideratges a les regions policials de Barcelona, Girona i la Central
2023-06-09
SCROLLING
SCROLLING
SCROLLING
new_height 18728
last_height 17474
SAME HEIGHTS
Detenim 3 homes i desmantellem a Constantí una plantació interior amb prop de 4.000 plantes de marihuana. 

Els detinguts vivien a l'interior de la nau des del març i no sortien per no aixecar sospites https://gen.cat/3qAdVmt
2023-06-10
Aquests dies ens han deixat el Pave i la Pretty, gossos de la Unitat Canina que han compartit servei amb nosaltres durant anys. La Pretty estava especialitzada en localització de drogues, armes i bitllets i el Pave en explosius. Es teus guies i nosaltres us trobarem a faltar
2023-06-09
Iniciem els primers canvis organitzatius de la nova estructura  que permetran desplegar el decret d’estructura d'una policia més moderna i tecnològica. Nous lide

 Som al #PAvConEurope2023, el congrés d'aviació policial més important d'Europa, on expliquem:

Els nostres mitjans aeris

 L'operativa i la tecnologia embarcada en els helicòpters

 Coordinació en les diferents operacions amb drons i helicòpters
2023-06-07
Avui comença 
@lapatum
 i nosaltres tindrem un dispositiu perquè la pugueu gaudir amb seguretat. Ens coordinarem amb la resta de cossos d'emergències d'
@interiorcat
 #Patum
2023-06-07
Molta sort a tots i totes els que us presenteu a les proves de Selectivitat #PAU23CAT!

Recorda que després d'estudiar a la universitat, tindràs l'oportunitat de venir a treballar amb nosaltres, busquem el teu talent #EnsFaltesTu
2023-06-07
SCROLLING
SCROLLING
SCROLLING
new_height 35062
last_height 34339
SAME HEIGHTS
Avui comença 
@lapatum
 i nosaltres tindrem un dispositiu perquè la pugueu gaudir amb seguretat. Ens coordinarem amb la resta de cossos d'emergències d'
@interiorcat
 #Patum
2023-06-07
Molta sort a tots i totes els que us presenteu a les 

 Supervisió i acompanyament de les famílies en l'ús d'internet
 Evitar la sobreexposició i el contingut inadequat

Són dos dels temes que vam tractar en la taula rodona "Acompanya'm a la xarxa", un projecte que hem ideat amb 
@granollers
2023-06-04
Despleguem un dispositiu a la plaça de Sant Jaume perquè pugueu gaudir de la celebració del títol de 
@UWCL
 aconseguit pel 
@FCBfemeni
 

Celebreu-ho amb respecte i seguretat #UWCLfinal #UWCL
2023-06-04
SCROLLING
SCROLLING
SCROLLING
new_height 40260
last_height 39412
SAME HEIGHTS
Si ets víctima de #violènciamasclista, domèstica o d’algun fet motivat per l’odi o la discriminació, pots escriure'ns un correu o un Whatsapp. T’assessorarem i resoldrem els teus dubtes. En cas de fer denúncia, farem seguiment del teu cas #25AnysGAV
2023-06-05
 Supervisió i acompanyament de les famílies en l'ús d'internet
 Evitar la sobreexposició i el contingut inadequat

Són dos dels temes que vam tractar en la taula rodona "Acompanya'm a la xarxa", un projecte q

KeyboardInterrupt: 

In [24]:
df

[('Mossos',
  "Si t'han insultat, agredit, discriminat o maltractat per la teva identitat o orientació sexual, denuncia-ho. T'escoltem. També pots escriure'ns a   mossos.delictes.odi@gencat.cat https://gen.cat/3rCR8mQ #StopLGTBIfòbia",
  '6',
  '2023-06-12T06:33:11.000Z'),
 ('Mossos',
  'Trobem un xai, que feia pocs dies que havia nascut, atrapat en uns matolls al riu Congost a Granollers. El recuperem i el lliurem al seu pastor',
  '22',
  '2023-06-11T18:51:18.000Z'),
 ('Mossos',
  "Avui, últim dia de \n@lapatum\n, un dels dies forts, i nosaltres seguim amb el dispositiu de seguretat. Gaudeix de la festa amb civisme i, si condueixes, recorda que l'única taxa segura és 0,0 #Patum",
  '6',
  '2023-06-11T17:03:25.000Z'),
 ('Mossos',
  'En el dispositiu ens coordinem amb Policia Local de \n@AjuntamentBerga\n des d’un nou centre de comandament avançat mòbil que s’estrena per l’ocasió',
  '2',
  '2023-06-11T18:29:15.000Z'),
 ('Mossos',
  "Per combatre la #violènciamasclista hem de tenir con

#### Saving tweets

In [25]:
with open('new_dataset_tweets_mossos.csv', 'a', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Text', 'RetweetCount','Timestamp']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(df)

### OFFICIAL ACCOUNTS - SEM. GENERALITAT

#### Searching

In [26]:
# Search "semgeneralitat" in searching bar
mossos = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[data-testid = "SearchBox_Search_Input"]')))
mossos.send_keys('semgencat')
mossos.send_keys(Keys.ENTER)

In [27]:
# Click into "Personas" tab
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a'))).click()

In [28]:
# Click into "semgencat" account (first one)
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]'))).click()

#### Getting the info

In [29]:
def get_tweet_data(driver):
    tag = driver.find_element(By.CSS_SELECTOR,'div[data-testid = "User-Name"]').text
    # Manage publi
    if len(tag.split('\n')) == 4:
        UserName, handle, _, Date = tag.split('\n')
        timestamp = driver.find_element(By.CSS_SELECTOR, 'time[datetime]').get_attribute('datetime')
    else:
        UserName = " "
        timestamp = "0001-01-01T00:00:00.000Z"
    # To avoid erros for tweets that do not have text
    element = driver.find_elements(By.CSS_SELECTOR,'div[data-testid = "tweetText"]')
    if len(element):
        Text = element[0].text
    else:
        Text = " "
    RetweetCount = driver.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
    tweet = (UserName, Text, RetweetCount, timestamp)
    return tweet

#### Scroll

In [30]:
# Define los tiempos de pausa entre scrolls
scroll_pause_time_1 = 1
scroll_pause_time_2 = 2

# Define variables útiles
df = []
tweet_ids = set()

# Verifica si hemos llegado al final de la región de scroll rastreando la posición de scroll
last_height = driver.execute_script("return document.body.scrollHeight")

# Define una variable para contar los intentos de desplazamiento sin nuevos tweets
scroll_attempt = 0

while True:
    tweets = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')

    if len(tweets) == 0:
        scroll_attempt += 1
    else:
        scroll_attempt = 0

    # Verifica si se han extraído nuevos tweets
    if scroll_attempt >= 3:
        # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
        break

    for tweet in tweets:
        info = get_tweet_data(tweet)
        print(info[1])
        if info:
            # Obtiene la fecha del tweet
            tweet_date = datetime.strptime(info[3], '%Y-%m-%dT%H:%M:%S.%fZ').date()
            print(tweet_date)
            # Crea un identificador de tweet
            tweet_id = ''.join(info)
            # Para evitar listar los mismos tweets
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                df.append(info)

    # Scrolls helps the page to arrive to the following point where we are interested in extracting tweets again
    for _ in range(3):
            print("SCROLLING")
            driver.execute_script("window.scrollBy(0, 700);")
            time.sleep(scroll_pause_time_1)

    # Wait to load page
    time.sleep(scroll_pause_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    print("new_height", new_height)
    print("last_height", last_height)

    # Verifica si se ha llegado al final de la página
    if new_height == last_height:
        scroll_attempt += 1
        if scroll_attempt >= 3:
            # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
            print("ATTEMPTS FINISHED")
            break
        else:
            # Wait before scrolling again
            time.sleep(scroll_pause_time_2)
    else:
        last_height = new_height
        print("SAME HEIGHTS")
        scroll_attempt = 0

Els professionals de l'ACUT Bages-Berguedà són els primers professionals d'atenció primària que visiten el Centre de Coordinació Operativa de Catalunya (CECAT) a Reus, la sala de coordinació des d'on es gestionen els recursos assistencials del 
@semgencat
 de #CatalunyaCentral.
2023-06-12
 Desplegament conjunt del #SEM i de 
@CreuRojaCAT
, durant tota 
@lapatum
 de #Berga d’aquesta edició 2023, garantint la millor resposta a la urgència i emergència. 

#LaPatum2023
2023-06-11
 Dispositiu #SEM, última nit de  
@lapatum
 de #Berga de 2023.
A una de les imatges, Unai, Edurne i Fernando, dins la Unitat de Coordinació Operativa
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 12134
last_height 11724
SAME HEIGHTS
Els professionals de l'ACUT Bages-Berguedà són els primers professionals d'atenció primària que visiten el Centre de Coordinació Operativa de Catalunya (CECAT) a Reus, la sala de coordinació des d'on es gestionen els recursos assistencials del 
@semgencat
 de #CatalunyaCentral.
2

 Les convulsions són contraccions involuntàries i violentes dels músculs del cos.

#PrimersAuxilis: Saps com reaccionar en aquests casos? T'ho expliquem en el vídeo a continuació 
2023-06-10
Ja tens l'app 061 Salut Respon en el teu mòbil?

 Recorda que omplir el teu perfil amb la màxima quantitat d'informació possible pot ajudar-te si necessites demanar assistència sanitària.
2023-06-10
 El cinturó de seguretat també l’hem de portar cordat en autobusos i autocars que en porten

 Aquesta setmana coordinem una campanya de controls de transport escolar, de menors i viatgers

#cordatalavida #CapRisc
#SCT
2023-06-09
 A Catalunya en necessitem 1.000 donacions cada dia! Pots donar sang:

 Si peses més de 50 quilos
 Encara que hagis patit hepatitis abans dels 12 anys
 Encara que no estiguis en dejú
 Encara que tinguis el colesterol elevat

 Demana cita a http://donarsang.gencat.cat/es/
2023-06-09
SCROLLING
SCROLLING
SCROLLING
new_height 16864
last_height 15769
SAME HEIGHTS
 El cinturó de segur

KeyboardInterrupt: 

#### Saving tweets

In [31]:
with open('new_dataset_tweets_semgeneralitat.csv', 'a', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Text', 'RetweetCount', 'Timestamp']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(df)
    f.close()

### OFFICIAL ACCOUNTS - DIARI DE GIRONA

#### Searching

In [ ]:
# Search user in searching bar
mossos = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[data-testid = "SearchBox_Search_Input"]')))
mossos.send_keys('DiarideGirona')
mossos.send_keys(Keys.ENTER)

In [ ]:
# Click into "Personas" tab
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a'))).click()

In [ ]:
# Click into "semgencat" account (first one)
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]'))).click()

#### Getting the info

In [ ]:
def get_tweet_data(driver):
    tag = driver.find_element(By.CSS_SELECTOR,'div[data-testid = "User-Name"]').text
    # Manage publi
    if len(tag.split('\n')) == 4:
        UserName, handle, _, Date = tag.split('\n')
        timestamp = driver.find_element(By.CSS_SELECTOR, 'time[datetime]').get_attribute('datetime')
    else:
        UserName = " "
        timestamp = "0001-01-01T00:00:00.000Z"
    # To avoid erros for tweets that do not have text
    element = driver.find_elements(By.CSS_SELECTOR,'div[data-testid = "tweetText"]')
    if len(element):
        Text = element[0].text
    else:
        Text = " "
    RetweetCount = driver.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
    tweet = (UserName, Text, RetweetCount, timestamp)
    return tweet

#### Scroll

In [ ]:
# Define los tiempos de pausa entre scrolls
scroll_pause_time_1 = 1
scroll_pause_time_2 = 2

# Define variables útiles
df = []
tweet_ids = set()

# Verifica si hemos llegado al final de la región de scroll rastreando la posición de scroll
last_height = driver.execute_script("return document.body.scrollHeight")

# Define una variable para contar los intentos de desplazamiento sin nuevos tweets
scroll_attempt = 0

while True:
    tweets = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')

    if len(tweets) == 0:
        scroll_attempt += 1
    else:
        scroll_attempt = 0

    # Verifica si se han extraído nuevos tweets
    if scroll_attempt >= 3:
        # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
        break

    for tweet in tweets:
        info = get_tweet_data(tweet)
        print(info[1])
        print(type(info[3]))
        if info:
            # Obtiene la fecha del tweet
            tweet_date = datetime.strptime(info[3], '%Y-%m-%dT%H:%M:%S.%fZ').date()
            print(tweet_date)
            # Crea un identificador de tweet
            tweet_id = ''.join(info)
            # Para evitar listar los mismos tweets
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                df.append(info)

    # Scrolls helps the page to arrive to the following point where we are interested in extracting tweets again
    for _ in range(3):
            print("SCROLLING")
            driver.execute_script("window.scrollBy(0, 700);")
            time.sleep(scroll_pause_time_1)

    # Wait to load page
    time.sleep(scroll_pause_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    print("new_height", new_height)
    print("last_height", last_height)

    # Verifica si se ha llegado al final de la página
    if new_height == last_height:
        scroll_attempt += 1
        if scroll_attempt >= 3:
            # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
            print("ATTEMPTS FINISHED")
            break
        else:
            # Wait before scrolling again
            time.sleep(scroll_pause_time_2)
    else:
        last_height = new_height
        print("SAME HEIGHTS")
        scroll_attempt = 0

#### Saving tweets

In [ ]:
with open('new_dataset_tweets_DiariDeGirona.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if os.stat("dataset_tweets_DiariDeGirona.csv").st_size == 0:
        header = ['UserName', 'Text', 'RetweetCount', 'Timestamp']
        writer.writerow(header)
    writer.writerows(df)
    f.close()

### OFFICIAL ACCOUNTS - CATALUNYA INFORMACIÓ

#### Searching

In [32]:
# Search user in searching bar
catinformacio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[data-testid = "SearchBox_Search_Input"]')))
catinformacio.send_keys('Catinformacio')
catinformacio.send_keys(Keys.ENTER)

In [33]:
# Click into "Personas" tab
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a'))).click()

In [34]:
# Click into "semgencat" account (first one)
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]'))).click()

#### Getting the info

In [35]:
def get_tweet_data(driver):
    tag = driver.find_element(By.CSS_SELECTOR,'div[data-testid = "User-Name"]').text
    # Manage publi
    if len(tag.split('\n')) == 4:
        UserName, handle, _, Date = tag.split('\n')
        timestamp = driver.find_element(By.CSS_SELECTOR, 'time[datetime]').get_attribute('datetime')
    else:
        UserName = " "
        timestamp = "0001-01-01T00:00:00.000Z"
    # To avoid erros for tweets that do not have text
    element = driver.find_elements(By.CSS_SELECTOR,'div[data-testid = "tweetText"]')
    if len(element):
        Text = element[0].text
    else:
        Text = " "
    RetweetCount = driver.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
    tweet = (UserName, Text, RetweetCount, timestamp)
    return tweet

#### Scrolling

In [36]:
# Define los tiempos de pausa entre scrolls
scroll_pause_time_1 = 1
scroll_pause_time_2 = 2

# Define variables útiles
df = []
tweet_ids = set()

# Verifica si hemos llegado al final de la región de scroll rastreando la posición de scroll
last_height = driver.execute_script("return document.body.scrollHeight")

# Define una variable para contar los intentos de desplazamiento sin nuevos tweets
scroll_attempt = 0

while True:
    tweets = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')

    if len(tweets) == 0:
        scroll_attempt += 1
    else:
        scroll_attempt = 0

    # Verifica si se han extraído nuevos tweets
    if scroll_attempt >= 3:
        # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
        break

    for tweet in tweets:
        info = get_tweet_data(tweet)
        print(info[1])
        print(type(info[3]))
        if info:
            # Obtiene la fecha del tweet
            tweet_date = datetime.strptime(info[3], '%Y-%m-%dT%H:%M:%S.%fZ').date()
            print(tweet_date)
            # Crea un identificador de tweet
            tweet_id = ''.join(info)
            # Para evitar listar los mismos tweets
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                df.append(info)

    # Scrolls helps the page to arrive to the following point where we are interested in extracting tweets again
    for _ in range(3):
            print("SCROLLING")
            driver.execute_script("window.scrollBy(0, 700);")
            time.sleep(scroll_pause_time_1)

    # Wait to load page
    time.sleep(scroll_pause_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    print("new_height", new_height)
    print("last_height", last_height)

    # Verifica si se ha llegado al final de la página
    if new_height == last_height:
        scroll_attempt += 1
        if scroll_attempt >= 3:
            # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
            print("ATTEMPTS FINISHED")
            break
        else:
            # Wait before scrolling again
            time.sleep(scroll_pause_time_2)
    else:
        last_height = new_height
        print("SAME HEIGHTS")
        scroll_attempt = 0

SCROLLING
SCROLLING
SCROLLING
new_height 12803
last_height 724
SAME HEIGHTS
 #ÚltimaHora Canvis al govern d'Aragonès. En surten els consellers Josep Gonzàlez-Cambray i Juli Fernàndez. Seran substituïts per Anna Simó a Educació i Ester Capella a Territori. David Mascort rellevarà Teresa Jordà a Acció Climàtica

http://catradio.cat/catinfo
<class 'str'>
2023-06-12
 Us agraden les cireres?  Què us en sembla el preu? Des d'una finca d'Alcarràs fins a un súper de Barcelona, el preu del quilo es duplica. 
Ho expliquem en aquest capítol de "La teoria del cigne negre" 
<class 'str'>
2023-06-12
 Repassem el #MésLlegitCR  Les claus de l'augment dels atacs d'orques a vaixells
<class 'str'>
2023-06-12
 El #MésLlegitCR  Suterranya: la connexió Catalunya-Xile
<class 'str'>
2023-06-12
 I acabem el repàs amb la notícia que encapçala el rànquing del #MésLlegitCR  Vine a córrer amb nosaltres: Manual de la cursa dels 40 anys
<class 'str'>
2023-06-12
SCROLLING
SCROLLING
SCROLLING
new_height 13984
last_hei

Nicola Sturgeon, investigada per presumpte finançament irregular de l'SPN, ha d'aclarir on van anar a parar més de 600.000 euros rebuts en donacions. Se sospita que no s'haurien destinat a la campanya del referèndum, sinó a despeses ordinàries del partit

http://catradio.cat/catinfo
<class 'str'>
2023-06-11
L'exèrcit ucraïnès assegura haver recuperat una localitat del sud-est del país que estava en mans russes. Aquesta petita població seria el primer guany territorial des de l'inici de la contraofensiva a la regió del Donbass

http://catradio.cat/catinfo
<class 'str'>
2023-06-11
 Sèries, música i ara també mascaretes coreanes. Espanya és el tercer importador mundial de cosmètica d'aquest país asiàtic
<class 'str'>
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 19451
last_height 18634
SAME HEIGHTS
Nicola Sturgeon, investigada per presumpte finançament irregular de l'SPN, ha d'aclarir on van anar a parar més de 600.000 euros rebuts en donacions. Se sospita que no s'haurien destinat 

 Els perills dels microcrèdits concedits per empreses financeres: demanar 100 euros i acabar devent-ne 9000
<class 'str'>
2023-06-11
Sense un horitzó de pau a la vista, molts joves palestins opten per la lluita armada
<class 'str'>
2023-06-11
 Demà començaran a fer les analítiques als nens de #Bonmatí (Selva) per saber si l'activitat d'una empresa de reciclatge de bateries del poble els està perjudicant.

 Miraran si tenen restes de plom a la sang
<class 'str'>
2023-06-11
Una dona de 75 anys va morir ofegada ahir a la tarda a la cala Canyelles de Lloret de Mar, a la Selva. La van poder treure de l'aigua inconscient, però no es va poder fer res per salvar-li la vida

http://catradio.cat/catinfo
<class 'str'>
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 36288
last_height 35063
SAME HEIGHTS
Sense un horitzó de pau a la vista, molts joves palestins opten per la lluita armada
<class 'str'>
2023-06-11
 Demà començaran a fer les analítiques als nens de #Bonmatí (Selva) per saber si l'a

 La manca de pluges i l'augment de les temperatures, conseqüència del canvi climàtic que vivim, tenen repercussions evidents en l'alimentació arreu del planeta
<class 'str'>
2023-06-10
 La cosmètica coreana és més que una tendència i no para de créixer a Catalunya

 T'expliquem els motius del boom 
<class 'str'>
2023-06-10
La Jim guanya l'última edició del programa Eufòria de TV3

#CatalunyaMigdiaCapDeSetmana
<class 'str'>
2023-06-10
SCROLLING
SCROLLING
SCROLLING
new_height 43273
last_height 41523
SAME HEIGHTS
 La manca de pluges i l'augment de les temperatures, conseqüència del canvi climàtic que vivim, tenen repercussions evidents en l'alimentació arreu del planeta
<class 'str'>
2023-06-10
 La cosmètica coreana és més que una tendència i no para de créixer a Catalunya

 T'expliquem els motius del boom 
<class 'str'>
2023-06-10
La Jim guanya l'última edició del programa Eufòria de TV3

#CatalunyaMigdiaCapDeSetmana
<class 'str'>
2023-06-10
Quatre nens trobats vius 40 dies després que l

SCROLLING
SCROLLING
new_height 59589
last_height 58777
SAME HEIGHTS
 I acabem el repàs amb la notícia que encapçala el rànquing del #MésLlegitCR  Punxar rodes per eliminar els cotxes de les ciutats
<class 'str'>
2023-06-10
 Els petards fa dies que se senten per pobles i ciutats malgrat que queden encara dues setmanes per a la revetlla.

 A #Figueres , una plataforma denuncia aquesta situació i alerta dels perjudicis que la pirotècnia provoca a molts col·lectius. 
<class 'str'>
2023-06-10
 En 10 anys s'ha multiplicat per quatre el nombre d'homes que se sotmeten a una cirurgia o un tractament d'estètica
<class 'str'>
2023-06-10
Celebrem els 100 anys de l'
@EdJuventud
. Saps quin és el seu llibre de més èxit?
@Tintin
? "Heidi"? Algun de Stefan Zweig? No és cap d'aquests i va per la 43a edició 
<class 'str'>
2023-06-10
 Els 226 nens i nenes que hi ha censats al poble de Bonmatí, a la Selva, estan cridats a fer-se una analítica per saber si l'aire que respiren és de bona qualitat
<class 'st

...

Assagen un producte per allunyar els cabirols dels camps de fesols de Santa Pau.

 
@accioclimatica
 diu que el repel•lent ha donat bon resultat a les vinyes del Priorat i Penedès.

 Es creu que a la Garrotxa hi ha una Densitat de 5 cabirols per km2

http://catradio.cat/catinfo
<class 'str'>
2023-06-09
SCROLLING
SCROLLING
SCROLLING
new_height 80769
last_height 79877
SAME HEIGHTS
 El "nen prodigi" de les sardanes a Catalunya.

 Jordi Montanyà, de Solsona, només té 14 anys i ja ha compost una sardana per a cobla:
<class 'str'>
2023-06-09
.
@JARC_COAG
  ha respost a la crida que els 
@agentsruralscat
  han fet perquè els pagesos llaurin els camps on no preveuen produir perquè facin de tallafocs.
Diuen que hi ha una normativa europea que impedeix llaurar abans de l'1 de setembre.
https://ccma.cat/catradio/directe/catalunya-informacio/…
<class 'str'>
2023-06-09
Assagen un producte per allunyar els cabirols dels camps de fesols de Santa Pau.

 
@accioclimatica
 diu que el repel•lent ha donat

 Coneixes la Casa Bofarull? És una de les joies modernistes del país, situada al poble dels Pallaresos, a poca distància de Tarragona.

 Ens conviden a visitar-la, al programa de turisme "Un país per voltar"
<class 'str'>
2023-06-09
#ÚltimaHora Operació dels Mossos d'Esquadra en maxa per detenir una quarantena de membres dels 'Casuals', un grup radical d'aficionats del Barça. Els vinculen amb delictes com homicidis, tinença il·lícita d'armes o extorsió, entre d'altres

http://catradio.cat/catinfo
<class 'str'>
2023-06-09
 De ben petit volia tocar la trompeta, i ara que té 14 anys ha compost la seva primera sardana.   

 El Jordi Montanyà s'ha  convertit en el compositor de sardanes més jove de Catalunya.
<class 'str'>
2023-06-09
 La cosmètica coreana és més que una tendència i no para de créixer a Catalunya. L'any passat les exportacions cap al l'estat espanyol van pujar un 229%

Aquí t'expliquem les claus de l'èxit 
<class 'str'>
2023-06-08
SCROLLING
SCROLLING
SCROLLING
new_height 872

 La Casa Bofarull, al poble dels Pallaresos, convida a fer-hi una visita guiada i veure com una masia pairal del segle XIV es va transformar en una referència del modernisme. 

 Hi anem al programa de turisme "Un país per voltar"
<class 'str'>
2023-06-08
 Coneixes la Casa Bofarull? És una de les joies modernistes del país, situada al poble dels Pallaresos, a poca distància de Tarragona.

 Ens conviden a visitar-la, al programa de turisme "Un país per voltar"
<class 'str'>
2023-06-08
Els Bombers han trobat el cos d'un home calcinat dins del perímetre de l'incendi a la Pobla de Massaluca, a la Terra Alta. Tot apunta que es tracta d'un pagès que feia unes cremes agrícoles i ha causat el foc 

Més notícies territorials a:
http://catradio.cat/tarragona
<class 'str'>
2023-06-08
 @CH_Ebro
 nega que s'hagi malbaratat aigua dels pantans de la conca del Segre per produir electricitat i assegura que només s'ha fet servir la del reg i el cabal ecològic

 Ho ha assegurat als representants del 
@Gra

En Àlbum il·lustrat s'ha premiat "Hi havia una forma" de Gazhole&Cruschiform.
<class 'str'>
2023-06-08
En  Àlbum il·lustrat ha guanyat "Hi havia una forma", de Gazhole&Cruschiform.
<class 'str'>
2023-06-08
Una vintena de propietaris de finques del delta de l'Ebre presenten una demanda davant l'Audiència Nacional, contra l'Estat, per la seva inacció en el compliment dels deures legals per preservar la integritat del delta

Més notícies territorials a:
http://catradio.cat/tarragona
<class 'str'>
2023-06-08
Declaració institucional del ple extraordinari de #Badalona per condemnar la última agressió sexual a una veïna de 13 anys. El ple insta altres possibles víctimes a denunciar  aquests delictes http://catradio.cat/catinfo
<class 'str'>
2023-06-08
 El Servei Català de Trànsit ha resolt el problema que feia que no es cobressin multes als conductors estrangers
<class 'str'>
2023-06-08
SCROLLING
SCROLLING
SCROLLING


KeyboardInterrupt: 

#### Saving tweets

In [37]:
with open('new_dataset_tweets_catinformacio.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if os.stat("dataset_tweets_catinformacio.csv").st_size == 0:
        header = ['UserName', 'Text', 'RetweetCount', 'Timestamp']
        writer.writerow(header)
    writer.writerows(df)
    f.close()

## OFFICIAL ACCOUNTS - 324cat

#### Searching

In [67]:
# Search user in searching bar
catinformacio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[data-testid = "SearchBox_Search_Input"]')))
catinformacio.send_keys('324cat')
catinformacio.send_keys(Keys.ENTER)

In [68]:
# Click into "Personas" tab
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a'))).click()

In [69]:
# Click into "semgencat" account (first one)
personas = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/section/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]'))).click()

#### Getting the info

In [70]:
def get_tweet_data(driver):
    tag = driver.find_element(By.CSS_SELECTOR,'div[data-testid = "User-Name"]').text
    # Manage publi
    if len(tag.split('\n')) == 4:
        UserName, handle, _, Date = tag.split('\n')
        timestamp = driver.find_element(By.CSS_SELECTOR, 'time[datetime]').get_attribute('datetime')
    else:
        UserName = " "
        timestamp = "0001-01-01T00:00:00.000Z"
    # To avoid erros for tweets that do not have text
    element = driver.find_elements(By.CSS_SELECTOR,'div[data-testid = "tweetText"]')
    if len(element):
        Text = element[0].text
    else:
        Text = " "
    RetweetCount = driver.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
    tweet = (UserName, Text, RetweetCount, timestamp)
    return tweet

#### Scrolling

In [78]:
# Define los tiempos de pausa entre scrolls
scroll_pause_time_1 = 1
scroll_pause_time_2 = 2

# Define variables útiles
df = []
tweet_ids = set()

# Verifica si hemos llegado al final de la región de scroll rastreando la posición de scroll
last_height = driver.execute_script("return document.body.scrollHeight")

# Define una variable para contar los intentos de desplazamiento sin nuevos tweets
scroll_attempt = 0

while True:
    tweets = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')

    if len(tweets) == 0:
        scroll_attempt += 1
    else:
        scroll_attempt = 0

    # Verifica si se han extraído nuevos tweets
    if scroll_attempt >= 3:
        # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
        break

    for tweet in tweets:
        info = get_tweet_data(tweet)
        print(info[1])
        print(type(info[3]))
        if info:
            # Obtiene la fecha del tweet
            tweet_date = datetime.strptime(info[3], '%Y-%m-%dT%H:%M:%S.%fZ').date()
            print(tweet_date)
            # Crea un identificador de tweet
            tweet_id = ''.join(info)
            # Para evitar listar los mismos tweets
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                df.append(info)

    # Scrolls helps the page to arrive to the following point where we are interested in extracting tweets again
    for _ in range(3):
            print("SCROLLING")
            driver.execute_script("window.scrollBy(0, 700);")
            time.sleep(scroll_pause_time_1)

    # Wait to load page
    time.sleep(scroll_pause_time_1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    print("new_height", new_height)
    print("last_height", last_height)

    # Verifica si se ha llegado al final de la página
    if new_height == last_height:
        scroll_attempt += 1
        if scroll_attempt >= 3:
            # Si no se han extraído nuevos tweets después de 3 intentos, finaliza el bucle
            print("ATTEMPTS FINISHED")
            break
        else:
            # Wait before scrolling again
            time.sleep(scroll_pause_time_2)
    else:
        last_height = new_height
        print("SAME HEIGHTS")
        scroll_attempt = 0

#ÚltimaHora Pacte a Ripoll perquè no governi l'Aliança Catalana de Sílvia Orriols, d'extrema dreta https://ccma.cat/324/pacte-a-ripoll-perque-no-governi-lalianca-catalana-de-silvia-orriols-dextrema-dreta/noticia/3234708/…
<class 'str'>
2023-06-12
Pere Aragonès ha comparegut aquest dilluns al matí per anunciar els canvis al govern: "El motiu és que cal encarar els pròxims dos anys amb energies renovades"
<class 'str'>
2023-06-12
Els creueristes també troben que Barcelona és una ciutat massificada pel turisme
<class 'str'>
2023-06-12
SCROLLING
SCROLLING
SCROLLING
new_height 59208
last_height 58871
SAME HEIGHTS
#ÚltimaHora Pacte a Ripoll perquè no governi l'Aliança Catalana de Sílvia Orriols, d'extrema dreta https://ccma.cat/324/pacte-a-ripoll-perque-no-governi-lalianca-catalana-de-silvia-orriols-dextrema-dreta/noticia/3234708/…
<class 'str'>
2023-06-12
Pere Aragonès ha comparegut aquest dilluns al matí per anunciar els canvis al govern: "El motiu és que cal encarar els pròxims dos anys a

Les flautes més antigues del món estan fetes amb ossos d'ocell i tenen 12.000 anys
https://ccma.cat/324/les-flautes-mes-antigues-del-mon-estan-fetes-amb-ossos-docell-i-tenen-12000-anys/noticia/3234443/…
<class 'str'>
2023-06-12
Els Mossos investiguen una agressió la nit de dissabte en un vagó del metro de Barcelona, i per la qual l'Ajuntament de Barcelona ha activat el protocol contra l'LGTBI-fòbia
<class 'str'>
2023-06-12
SCROLLING
SCROLLING
SCROLLING
new_height 62007
last_height 60960
SAME HEIGHTS
Des de fa unes setmanes ja s'administra a Catalunya el Voxzogo, el nou medicament que permet a les persones que tenen acondroplàsia créixer cada any un centímetre i mig més
<class 'str'>
2023-06-12
Les flautes més antigues del món estan fetes amb ossos d'ocell i tenen 12.000 anys
https://ccma.cat/324/les-flautes-mes-antigues-del-mon-estan-fetes-amb-ossos-docell-i-tenen-12000-anys/noticia/3234443/…
<class 'str'>
2023-06-12
Els Mossos investiguen una agressió la nit de dissabte en un vagó del

Suspès del càrrec l'ambaixador de Romania a Kènia. Era en una reunió a Nairobi quan va veure un mico a l'altra banda de la finestra i va dir: "El grup africà ja se'ns ha unit"
<class 'str'>
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 64323
last_height 63387
SAME HEIGHTS
“Mai et penses que un fill tindrà càncer”. El Pol tenia dos anys quan li van detectar un càncer avançat. El van tractar a Sant Joan de Déu, l’únic centre especialitzat en càncer infantil d’Espanya. Va ser el primer al món a fer l’assaig d’una immunoteràpia https://tinyurl.com/223v9r2t
<class 'str'>
2023-06-11
Shannen Doherty, de "Sensación de vivir", revela que té metàstasi al cervell: "Estic aterrida" -
<class 'str'>
2023-06-11
Suspès del càrrec l'ambaixador de Romania a Kènia. Era en una reunió a Nairobi quan va veure un mico a l'altra banda de la finestra i va dir: "El grup africà ja se'ns ha unit"
<class 'str'>
2023-06-11
Abans i després de la destrucció: imatges des de l'aire de la devastació a la presa d'U

Els quatre germans perduts a la selva de l'Amazònia van poder sobreviure 40 dies a la intempèrie gràcies als coneixements sobre l'entorn de la germana gran, la Lesly
<class 'str'>
2023-06-11
Detenen Nicola Sturgeon, l'exprimera ministra d'Escòcia, per finançament irregular del partit 
http://ccma.cat/324/detenen-nicola-sturgeon-lexprimera-ministra-descocia-per-financament-irregular-del-partit/noticia/3234567…  via 
@324cat
<class 'str'>
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 69537
last_height 68577
SAME HEIGHTS
Consum investiga set aerolínies per fer pagar per l'equipatge de mà i per triar seient
<class 'str'>
2023-06-11
Els Mossos detenen tres lladres que van robar un rellotge valorat en mig milió d'euros a un home quan sortia d'un restaurant de luxe
<class 'str'>
2023-06-11
Els quatre germans perduts a la selva de l'Amazònia van poder sobreviure 40 dies a la intempèrie gràcies als coneixements sobre l'entorn de la germana gran, la Lesly
<class 'str'>
2023-06-11
Detenen N

La dieta dels rincosaures, basada en plantes dures, desgastava les seves dents fins que, quan eren grans, els científics creuen que morien de gana per la dificultat d’alimentar-se
<class 'str'>
2023-06-11
Tenir unes vacances ja contractades permetrà no ser a la mesa electoral el 23J
<class 'str'>
2023-06-11
Fer mindfulness durant l'embaràs, un aliat per reduir preeclàmpsies i baix pes del nadó
<class 'str'>
2023-06-11
SCROLLING
SCROLLING
SCROLLING
new_height 71717
last_height 70932
SAME HEIGHTS
La dieta dels rincosaures, basada en plantes dures, desgastava les seves dents fins que, quan eren grans, els científics creuen que morien de gana per la dificultat d’alimentar-se
<class 'str'>
2023-06-11
Tenir unes vacances ja contractades permetrà no ser a la mesa electoral el 23J
<class 'str'>
2023-06-11
Fer mindfulness durant l'embaràs, un aliat per reduir preeclàmpsies i baix pes del nadó
<class 'str'>
2023-06-11
Avís de tempestes fortes. La méteo del 
@324cat
 amb 
@enricagud
 http://short

...

SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin decidit sobreviure amagant-se
<class 'str'>
2023-06-03
Consulta les dates clau de les eleccions generals al Congrés dels Diputats i al Senat que se celebraran el pròxim 23 de juliol
<class 'str'>
2023-06-03
La forta concentració bancària a Espanya perjudica els estalviadors i explica per què els bancs paguen poc pels dipòsits
<class 'str'>
2023-06-03
Un estudi de l'Hospital Germans Trias i Pujol detecta que les persones amb migranya tenen alteracions cerebrals visibles, cosa que pot facilitar el diagnòstic
<class 'str'>
2023-06-03
SCROLLING
SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin dec

SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin decidit sobreviure amagant-se
<class 'str'>
2023-06-03
Consulta les dates clau de les eleccions generals al Congrés dels Diputats i al Senat que se celebraran el pròxim 23 de juliol
<class 'str'>
2023-06-03
La forta concentració bancària a Espanya perjudica els estalviadors i explica per què els bancs paguen poc pels dipòsits
<class 'str'>
2023-06-03
Un estudi de l'Hospital Germans Trias i Pujol detecta que les persones amb migranya tenen alteracions cerebrals visibles, cosa que pot facilitar el diagnòstic
<class 'str'>
2023-06-03
SCROLLING
SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin dec

SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin decidit sobreviure amagant-se
<class 'str'>
2023-06-03
Consulta les dates clau de les eleccions generals al Congrés dels Diputats i al Senat que se celebraran el pròxim 23 de juliol
<class 'str'>
2023-06-03
La forta concentració bancària a Espanya perjudica els estalviadors i explica per què els bancs paguen poc pels dipòsits
<class 'str'>
2023-06-03
Un estudi de l'Hospital Germans Trias i Pujol detecta que les persones amb migranya tenen alteracions cerebrals visibles, cosa que pot facilitar el diagnòstic
<class 'str'>
2023-06-03
SCROLLING
SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin dec

SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin decidit sobreviure amagant-se
<class 'str'>
2023-06-03
Consulta les dates clau de les eleccions generals al Congrés dels Diputats i al Senat que se celebraran el pròxim 23 de juliol
<class 'str'>
2023-06-03
La forta concentració bancària a Espanya perjudica els estalviadors i explica per què els bancs paguen poc pels dipòsits
<class 'str'>
2023-06-03
Un estudi de l'Hospital Germans Trias i Pujol detecta que les persones amb migranya tenen alteracions cerebrals visibles, cosa que pot facilitar el diagnòstic
<class 'str'>
2023-06-03
SCROLLING
SCROLLING
SCROLLING
new_height 426563
last_height 426563
Els familiars dels menors desapareguts en l’accident asseguren que els nens coneixen bé la selva. Els preocupa, però, que els menors pensin que ningú els busca i hagin dec

KeyboardInterrupt: 

#### Saving tweets

In [79]:
with open('new_dataset_tweets_324cat.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if os.stat("new_dataset_tweets_324cat.csv").st_size == 0:
        header = ['UserName', 'Text', 'RetweetCount', 'Timestamp']
        writer.writerow(header)
    writer.writerows(df)
    f.close()

#### CLOSE DRIVER

In [58]:
driver.quit()